In [1]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import StringType, IntegerType, FloatType, StructType, StructField, TimestampType, DateType, ArrayType, LongType
import datetime
import pandas as pd

In [2]:
sc

<SparkContext master=yarn appName=PySparkShell>

In [3]:
sqlContext = SQLContext(sc)

In [4]:
def loadData(loc):
    df = sqlContext.read.format("csv") \
    .option("header", "true") \
    .option("timestampFormat", "MM/dd/yyyy HH:mm:SS") \
    .option("inferSchema", "true") \
    .load(loc)
    
    return df

def writeOutput(df, loc):
    df.repartition(1).write.option("header", "true").csv(loc)

In [5]:
bucket = 'gs://project_ggallagher6/' #201410-citibike-tripdata.csv'

weatherLoc = bucket+'wx_data_noaa_cleaned.csv'
df = loadData(weatherLoc)
df.show(5)

+-------------------+-------------------+---------------+--------+------+----------+----------+--------+--------+---+-------+---+----+----------+--------+---------+-------+----+----+
|               DATE|           NEW_DATE|    DATE_STRING|AVG_WIND|PRECIP|SNOW_TOTAL|SNOW_DEPTH|MAX_TEMP|MIN_TEMP|FOG|THUNDER|ICE|HAIL|HAZE_SMOKE|BLW_SNOW|HIGH_WIND|DRIZZLE|RAIN|SNOW|
+-------------------+-------------------+---------------+--------+------+----------+----------+--------+--------+---+-------+---+----+----------+--------+---------+-------+----+----+
|2010-01-01 00:00:00|2010-01-01 00:00:00|01 January 2010|    3.36|  0.03|       0.0|       0.0|      40|      33|Yes|     No| No|  No|        No|      No|       No|     No| Yes| Yes|
|2010-01-02 00:00:00|2010-01-02 00:00:00|02 January 2010|   12.53|  0.02|       0.2|       0.0|      34|      17|Yes|     No| No|  No|       Yes|      No|       No|     No| Yes| Yes|
|2010-01-03 00:00:00|2010-01-03 00:00:00|03 January 2010|   14.32|   0.0|       0.0| 

In [6]:
weather = df.withColumn('weatherHour', hour(df.DATE)).withColumn('weatherDate', to_date(df.DATE))
#weather.printSchema()
weatherMin = weather.drop('DATE', 'NEW_DATE', 'DATE_STRING')
weatherMin.show(5)

+--------+------+----------+----------+--------+--------+---+-------+---+----+----------+--------+---------+-------+----+----+-----------+-----------+
|AVG_WIND|PRECIP|SNOW_TOTAL|SNOW_DEPTH|MAX_TEMP|MIN_TEMP|FOG|THUNDER|ICE|HAIL|HAZE_SMOKE|BLW_SNOW|HIGH_WIND|DRIZZLE|RAIN|SNOW|weatherHour|weatherDate|
+--------+------+----------+----------+--------+--------+---+-------+---+----+----------+--------+---------+-------+----+----+-----------+-----------+
|    3.36|  0.03|       0.0|       0.0|      40|      33|Yes|     No| No|  No|        No|      No|       No|     No| Yes| Yes|          0| 2010-01-01|
|   12.53|  0.02|       0.2|       0.0|      34|      17|Yes|     No| No|  No|       Yes|      No|       No|     No| Yes| Yes|          0| 2010-01-02|
|   14.32|   0.0|       0.0|       0.0|      22|      17| No|     No| No|  No|       Yes|      No|       No|     No|  No| Yes|          0| 2010-01-03|
|   10.74|   0.0|       0.0|       0.0|      30|      19| No|     No| No|  No|        No|     

In [8]:
data = loadData(bucket+'2014_demand.csv')
stations = loadData(bucket+'2014_stations.csv')

In [32]:
stationList = list(stations.select('station_id').toPandas()['station_id'])
print(len(stationList))

data.printSchema()
data.show(2)

#data2016Clean = data2016.withColumn('date', to_date(data2016.date))
#test = data2016Clean.filter(data2016Clean.start_station_id == 492)
#test2 = test.groupBy('date','hour','start_station_id').count()

#https://stackoverflow.com/questions/46709285/filling-missing-dates-in-spark-dataframe-column
#test2 = test2.orderBy(test2.date, test2.hour, ascending=True)
#test2.show(5)
#data2015Clean.show(5)
#data2015Clean.printSchema()

345
root
 |-- date: timestamp (nullable = true)
 |-- hour: integer (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- count: integer (nullable = true)

+-------------------+----+----------------+-----+
|               date|hour|start_station_id|count|
+-------------------+----+----------------+-----+
|2014-01-01 00:00:00|   2|             157|    1|
|2014-01-01 00:00:00|   4|             439|    1|
+-------------------+----+----------------+-----+
only showing top 2 rows



In [ ]:
#test2.show(5)
for i in range(len(stationList)):
    station = stationList[i]
    stationDemand = data.filter(data.start_station_id == station)
    stationDemandDF = stationDemand.toPandas()
    stationDemandDF['date'] = pd.to_datetime(stationDemandDF['date'])
    stationDemandDF = stationDemandDF.astype({'hour':'int64', 'start_station_id': 'int64'})
    r = pd.date_range(start='2014-01-01', end=stationDemandDF.date.max())
    #print(r)
    h = []
    for i in range(24):
        h.append(i)
    
    iterables = [r, h]

    new_index = pd.MultiIndex.from_product(iterables, names=["d", "h"])
    #print(bob)
    df = stationDemandDF.set_index(['date','hour']).reindex(new_index).fillna(0).rename_axis(['date', 'hour']).reset_index()
    df['start_station_id'] = stationList[0]
    tst = tst.astype({'count':'int64'})
    
    weatherDF = weatherMin.toPandas()
    weatherDF['weatherDate'] = pd.to_datetime(weatherDF['weatherDate'])
    bob = df.merge(weatherDF, how='left', left_on='date', right_on='weatherDate')
    bob.to_csv(bucket+'stationDemand/2014_'+str(station)+'.csv', index=False)
    #test = data2019Clean.join(weatherMin,(data2019Clean.date == weatherMin.weatherDate), how='left')
    
    
#test = data.filter(data.start_station_id == stationList[0])
#test = test.orderBy(test.date, test.hour, ascending=True)
#test.show(5)
if False:
    testDF = test.toPandas()
    testDF['date'] = pd.to_datetime(testDF['date'])
    testDF = testDF.astype({'hour':'int64', 'start_station_id': 'int64'})
    r = pd.date_range(start='2014-01-01', end=testDF.date.max())
    #print(r)
    h = []
    for i in range(24):
        h.append(i)
#print(h)
#testDF.set_index('date').reindex(r).fillna(0.0).rename_axis('date').reset_index()
#df_filled = df.groupby('date', 'hour').apply(lambda group: group.fillna(value=0))
#print(testDF)h
#print(r)
#print(testDF)

    iterables = [r, h]

    bob = pd.MultiIndex.from_product(iterables, names=["d", "h"])
    #print(bob)
    tst = testDF.set_index(['date','hour']).reindex(bob).fillna(0).rename_axis(['date', 'hour']).reset_index()
    tst['start_station_id'] = stationList[0]
    tst = tst.astype({'count':'int64'})
    display(tst)


In [29]:
#print(testDF.info())
#tim = testDF.copy(deep=True)
#tim['date'] = pd.to_datetime(tim['date'])
#tim = tim.astype({'hour':'int64', 'start_station_id': 'int64'})
#print(tim.info())
#df.astype({'col1': 'int32'})
#print(tim)
#print(tst.info())
#tst.to_csv(bucket+'tacos/bob.csv')
#bucket+'test/'
print(str(len(stationList)))
#stationList = stationList.sort()
print(stationList)

TypeError: object of type 'NoneType' has no len()

In [92]:
weatherDF = weatherMin.toPandas()
weatherDF['weatherDate'] = pd.to_datetime(weatherDF['weatherDate'])
bob = tst.merge(weatherDF, how='left', left_on='date', right_on='weatherDate')
print(weatherDF.info())
print(bob)
bob.to_csv(bucket+'2016_492.csv', index=False)
#test = data2019Clean.join(weatherMin,(data2019Clean.date == weatherMin.weatherDate), how='left')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4307 entries, 0 to 4306
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   AVG_WIND     4307 non-null   object        
 1   PRECIP       4307 non-null   float64       
 2   SNOW_TOTAL   4307 non-null   float64       
 3   SNOW_DEPTH   4306 non-null   float64       
 4   MAX_TEMP     4307 non-null   int32         
 5   MIN_TEMP     4307 non-null   int32         
 6   FOG          4307 non-null   object        
 7   THUNDER      4307 non-null   object        
 8   ICE          4307 non-null   object        
 9   HAIL         4307 non-null   object        
 10  HAZE_SMOKE   4307 non-null   object        
 11  BLW_SNOW     4307 non-null   object        
 12  HIGH_WIND    4307 non-null   object        
 13  DRIZZLE      4307 non-null   object        
 14  RAIN         4307 non-null   object        
 15  SNOW         4307 non-null   object        
 16  weathe

In [ ]:
test2.printSchema()
bob = test2.collect()[0][0]
print(bob)
#datetime.datetime.strptime('24052010', "%d%m%Y").date()
# The size of each step in days https://stackoverflow.com/questions/46709285/filling-missing-dates-in-spark-dataframe-column

day_delta = datetime.timedelta(days=1)

start_date = datetime.datetime.strptime('01012015', "%d%m%Y").date()
end_date = start_date + 365*day_delta
columns = ['date', 'hour', 'start_station_id', 'count']

for i in range((end_date - start_date).days):
    for j in range(24):
        #    print(start_date + i*day_delta)
        print("trying " + str(start_date + i*day_delta) + " and " + str(j))
        test3 = test2.filter((test2.date.cast(StringType()) == str(start_date + i*day_delta)) & (test2.hour == j))
        if test3.count() == 0:
            newDF = spark.createDataFrame([(start_date + i*day_delta, j, 492, 0)], columns)
            test2 = test2.union(newDF)
            
print(str(test2.count()))

root
 |-- date: date (nullable = true)
 |-- hour: integer (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- count: long (nullable = false)

2015-04-01
trying 2015-01-01 and 0
trying 2015-01-01 and 1
trying 2015-01-01 and 2
trying 2015-01-01 and 3
trying 2015-01-01 and 4
trying 2015-01-01 and 5
trying 2015-01-01 and 6
trying 2015-01-01 and 7
trying 2015-01-01 and 8
trying 2015-01-01 and 9
trying 2015-01-01 and 10
trying 2015-01-01 and 11
trying 2015-01-01 and 12
trying 2015-01-01 and 13
trying 2015-01-01 and 14


KeyboardInterrupt: 

In [12]:
test = data2019Clean.join(weatherMin,(data2019Clean.date == weatherMin.weatherDate), how='left')
test.show(2)
test.drop('weatherDate', 'weatherHour')
writeOutput(test, bucket+'test/')

+----------+----+----------------+--------------+-----+--------+------+----------+----------+--------+--------+---+-------+---+----+----------+--------+---------+-------+----+----+-----------+-----------+
|      date|hour|start_station_id|end_station_id|count|AVG_WIND|PRECIP|SNOW_TOTAL|SNOW_DEPTH|MAX_TEMP|MIN_TEMP|FOG|THUNDER|ICE|HAIL|HAZE_SMOKE|BLW_SNOW|HIGH_WIND|DRIZZLE|RAIN|SNOW|weatherHour|weatherDate|
+----------+----+----------------+--------------+-----+--------+------+----------+----------+--------+--------+---+-------+---+----+----------+--------+---------+-------+----+----+-----------+-----------+
|2019-01-01|   1|             326|           301|    1| Unknown|  0.06|       0.0|       0.0|      58|      39|Yes|     No| No|  No|        No|      No|       No|     No|  No|  No|          0| 2019-01-01|
|2019-01-01|   2|             447|           461|    1| Unknown|  0.06|       0.0|       0.0|      58|      39|Yes|     No| No|  No|        No|      No|       No|     No|  No|  No|

KeyboardInterrupt: 

In [12]:
print(data2014Clean.count())
print(test.count())

7185729
7185729


In [ ]:
for i in range(2014, 2020):
    data = loadData(bucket+'cleaned/' + str(i) +'.csv')
    
    out = data.join(weatherMin,(data.date == weatherMin.weatherDate), how='left')
    out = out.drop('weatherDate', 'weatherHour')

    writeOutput(out, bucket+"cleanWeather2/" + str(i))